In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import numpy as np
import simpy
import simpy.events as evt
import random

In [2]:
#Initialize data of list 

data =  {'Line':['HS2'], 'From':['London Euston'], 
        'To': ['London Old Oak Common'], 'Distance (m)': [145000],
        'Time': [1740], 'Delay': [300], 'Signaling Blocks': [10]}

#Create Data frame 

df = pd.DataFrame(data) 
df

,Line,From,To,Distance (m),Time,Delay,Signaling Blocks
0,HS2,London Euston,London Old Oak Common,145000,1740,300,10


In [3]:
#Record all Simulation result 

results = pd.DataFrame({'departure':[],'arrival':[],'passengers':[],'travel_time':[]})
results

,departure,arrival,passengers,travel_time


In [4]:
#Prepare customers data using Poisson Distribution

def poissonEvents(λ=None, N=None, T=None, 
                  plot=True, events=None, figsize=None):

    if events!=None:
        N = len(events)
        
    if N!=None and T==None and λ!=None:
        T = int(N/λ)
    elif N==None and T!=None and λ!=None:
        N = int(λ*T)
    elif N!=None and T!=None and λ==None:
        λ = N/T
    
   
    if events==None:
        u = [ random.uniform(0, 1) for i in range(N) ]
        P = list(np.cumsum(list(map(
                lambda x: -math.log(1-x)/λ, u))))
    else:
        P = events
        
    if plot:
  
        if figsize!=None:
            width, height = figsize
        else:
            width, height = 10, 6
            
        fig, ax=plt.subplots(1,1)
        fig.set_figwidth(width)
        fig.set_figheight(height)
        
        def X(l):
            
            xmax = max(int(N/λ),math.ceil(max(l)))
            
            def double(l):
                return [] if l==[] \
                        else [l[0], l[0]]+double(l[1:])
    
            return [0]+double(l)+[xmax]
        
        def Y(l):
    
            def steps(l, n):
                return [] if l==[] \
                        else [n, n]+steps(l[1:], n+1)
    
            return [0, 0]+steps(l, 1)
    
        x = X(P)
        y = Y(P)

        ax.set_title(f"Poisson Process λ={λ:5.3f} n={N:d}")
        ax.set_xlim(min(x), max(x))
        ax.set_ylim(min(y), max(y))
        ax.yaxis.set_major_locator(
            mpl.ticker.MaxNLocator(integer=True))
        ax.plot(x, y, lw=3)
        ax.plot(x, list(map(lambda x:λ*x, x)))
        ax.grid(True)
        
    return P

In [5]:
#Define function that calls the poisson event to generate Customer data
# we set λ=100/3600 to get a maximum of 100 customers per hour 
# In total we generate 300 passengers as N is set to 100 for each stations 
def Pass():
    
    passengers_euston = poissonEvents(λ=100/3600, N=100000, plot=False) 
    euston = pd.DataFrame({'station':['London Euston']*len(passengers_euston), 'arr':passengers_euston})
    passengers= euston
    return passengers

In [6]:
class Train:  
    def __init__(self, i, route, d): # maxPassengers=400
        self.name ='[Train '+f"{i%20:2d}"+']'
        self.route = route
        self.r = i
        self.p     = 0
       # self.maxP  = maxPassengers
        self.blocks = d
    
    def process(self):
        
        print(f"{now():s} {self.name:s} drives to starting location")
        if now()=="09:00:00":
            yield env.timeout(1800)
            print("Technical Fault")
        else:    
            yield env.timeout(60)
        startingstation = self.route[0]   # starting location
        i=0
        results.at[self.r,'departure'] = env.now
        num = board(startingstation,env.now)
        self.p += num
        print(f"{now():s} {self.name:s} {num} passengers boarded")
        yield env.timeout(num*3) # num is the total number of passengers boarded and 
        #each passengers take 3 seconds to get on board 
        
        print(f"{now():s} {self.name:s} dep {startingstation:s}")
        for dest in self.route[1:]:
            data=df[df['From']==startingstation]
            travelTime=data.iloc[0].at['Time']
            sig=data.iloc[0].at['Signaling Blocks']
            delay=data.iloc[0].at['Delay']
            blocks = self.blocks[i]
            block_travel = travelTime/sig
            
            l = 1
            #yield env.timeout(delay)
            
            for block in blocks:
                with block.request() as req:
                    yield req
                    
                    print(f"{now():s} {self.name:s} enters block {l:d} for {dest:s}") 
                    
                    yield env.timeout(block_travel)

                    if l == len(blocks):
                        print(f"{now():s} {self.name:s} arr {dest:s}")
                        num = board(dest,env.now)
                        self.p += num
                        print(f"{now():s} {self.name:s} {num} passengers boarded")
                        yield env.timeout(num*3)
                        startingstation=dest
                        
                    if startingstation == "London Old Oak Common":
                        print(f"{now():s} {self.name:s} arr final station") #no passengers will get on board at the final station
                        results.at[self.r,'arrival'] = env.now
                        results.at[self.r,'passengers'] = self.p
                        results.at[self.r,'travel_time'] = results.at[self.r,'arrival']-results.at[self.r,'departure']
                    
                l+=1         
                    
            i+=1
          
   
    

In [7]:
#Number of passengers at the station when the train arrives 

def board(station,s):
    
    df = passengers[passengers['station'] == station]
    t = df['arr']
    g = s-21600
    passengercount = 0
    for arr in t:
        if arr>0 and arr<=g:
            passengercount+=1
            passengers.arr = passengers.arr.replace(arr,0)
    
    return passengercount
            

In [8]:
def block():
    blocks = []
    stations = df['From'].to_list()
    stations += [df['To'].to_list()[-1]]
    startingBlock = stations[0] # STARTING LOCATION
    for dest in stations[1:]:
        data = df[df['From'] == startingBlock]
        sb = data.iloc[0].at['Signaling Blocks']
        destb = []
        for i in range(sb):
            block = simpy.Resource(env, capacity = 1)
            destb.append(block)
        blocks.append(destb)
        startingBlock = dest 
    return blocks

In [9]:
#Instantiate the train class, here we define class that calls and release the trains for every stations 

def line(d, start=6*3600, stop=22*3600, timing=180):
    stations=df['From'].to_list()
    stations+=[df['To'].to_list()[-1]]
    yield env.timeout(start-env.now) # the line starts operating at 6am
    for i in range(int((stop-start)/timing)):
        t=Train(i, stations, d)
        env.process(t.process())
        yield env.timeout(timing)

In [10]:
#format Date and Time 
def daytime(t):
    t=int(t)
    return f"{t//3600:02d}:{(t%3600)//60:02d}:{t%60:02d}"
def now():
    return daytime(env.now)

In [11]:
def runsimulation(a,b):
    global passengers
    global env
    global results
    
    df.iloc[0,6]=b
    passengers= Pass()
    env = simpy.Environment()
    
    d = block() 
    env.process(line(d))
    env.run()
    
    return results
    
    

In [12]:
def stats(results):
    
    return np.mean(results['travel_time']), np.sum(results['passengers'])
    

In [13]:
runsimulation(20,14) 

06:00:00 [Train  0] drives to starting location
06:01:00 [Train  0] 0 passengers boarded
06:01:00 [Train  0] dep London Euston
06:01:00 [Train  0] enters block 1 for London Old Oak Common
06:03:00 [Train  1] drives to starting location
06:03:04 [Train  0] enters block 2 for London Old Oak Common
06:04:00 [Train  1] 6 passengers boarded
06:04:18 [Train  1] dep London Euston
06:04:18 [Train  1] enters block 1 for London Old Oak Common
06:05:08 [Train  0] enters block 3 for London Old Oak Common
06:06:00 [Train  2] drives to starting location
06:06:22 [Train  1] enters block 2 for London Old Oak Common
06:07:00 [Train  2] 6 passengers boarded
06:07:12 [Train  0] enters block 4 for London Old Oak Common
06:07:18 [Train  2] dep London Euston
06:07:18 [Train  2] enters block 1 for London Old Oak Common
06:08:26 [Train  1] enters block 3 for London Old Oak Common
06:09:00 [Train  3] drives to starting location
06:09:17 [Train  0] enters block 5 for London Old Oak Common
06:09:22 [Train  2] en

06:40:00 [Train 13] 5 passengers boarded
06:40:13 [Train  4] enters block 14 for London Old Oak Common
06:40:15 [Train 13] dep London Euston
06:40:15 [Train 13] enters block 1 for London Old Oak Common
06:40:24 [Train 11] enters block 4 for London Old Oak Common
06:40:34 [Train  9] enters block 7 for London Old Oak Common
06:40:50 [Train  7] enters block 10 for London Old Oak Common
06:41:12 [Train  5] enters block 13 for London Old Oak Common
06:41:33 [Train 10] enters block 6 for London Old Oak Common
06:41:35 [Train 12] enters block 3 for London Old Oak Common
06:41:46 [Train  8] enters block 9 for London Old Oak Common
06:41:59 [Train  6] enters block 12 for London Old Oak Common
06:42:00 [Train 14] drives to starting location
06:42:17 [Train  4] arr London Old Oak Common
06:42:17 [Train  4] 0 passengers boarded
06:42:17 [Train  4] arr final station
06:42:19 [Train 13] enters block 2 for London Old Oak Common
06:42:29 [Train 11] enters block 5 for London Old Oak Common
06:42:38 [Tr

07:10:00 [Train  3] 6 passengers boarded
07:10:07 [Train 14] enters block 14 for London Old Oak Common
07:10:18 [Train  3] dep London Euston
07:10:18 [Train  3] enters block 1 for London Old Oak Common
07:10:39 [Train  1] enters block 4 for London Old Oak Common
07:10:52 [Train 19] enters block 7 for London Old Oak Common
07:10:56 [Train 17] enters block 10 for London Old Oak Common
07:11:12 [Train 15] enters block 13 for London Old Oak Common
07:11:27 [Train  0] enters block 6 for London Old Oak Common
07:11:29 [Train  2] enters block 3 for London Old Oak Common
07:11:46 [Train 18] enters block 9 for London Old Oak Common
07:11:56 [Train 16] enters block 12 for London Old Oak Common
07:12:00 [Train  4] drives to starting location
07:12:11 [Train 14] arr London Old Oak Common
07:12:11 [Train 14] 0 passengers boarded
07:12:11 [Train 14] arr final station
07:12:22 [Train  3] enters block 2 for London Old Oak Common
07:12:44 [Train  1] enters block 5 for London Old Oak Common
07:12:56 [Tr

07:43:00 [Train 14] 1 passengers boarded
07:43:03 [Train 14] dep London Euston
07:43:03 [Train 14] enters block 1 for London Old Oak Common
07:43:03 [Train  7] enters block 11 for London Old Oak Common
07:43:19 [Train  5] enters block 14 for London Old Oak Common
07:43:33 [Train 12] enters block 4 for London Old Oak Common
07:43:34 [Train 10] enters block 7 for London Old Oak Common
07:43:50 [Train  8] enters block 10 for London Old Oak Common
07:44:03 [Train  6] enters block 13 for London Old Oak Common
07:44:29 [Train 13] enters block 3 for London Old Oak Common
07:44:40 [Train  9] enters block 9 for London Old Oak Common
07:44:45 [Train 11] enters block 6 for London Old Oak Common
07:45:00 [Train 15] drives to starting location
07:45:07 [Train 14] enters block 2 for London Old Oak Common
07:45:08 [Train  7] enters block 12 for London Old Oak Common
07:45:23 [Train  5] arr London Old Oak Common
07:45:23 [Train  5] 0 passengers boarded
07:45:23 [Train  5] arr final station
07:45:38 [T

08:06:20 [Train 12] 0 passengers boarded
08:06:20 [Train 12] arr final station
08:06:22 [Train  1] enters block 2 for London Old Oak Common
08:06:41 [Train 17] enters block 8 for London Old Oak Common
08:06:42 [Train 15] enters block 11 for London Old Oak Common
08:06:47 [Train 19] enters block 5 for London Old Oak Common
08:07:00 [Train  2] 12 passengers boarded
08:07:16 [Train 13] enters block 14 for London Old Oak Common
08:07:30 [Train  0] enters block 4 for London Old Oak Common
08:07:31 [Train 18] enters block 7 for London Old Oak Common
08:07:36 [Train  2] dep London Euston
08:07:36 [Train  2] enters block 1 for London Old Oak Common
08:07:53 [Train 16] enters block 10 for London Old Oak Common
08:07:54 [Train 14] enters block 13 for London Old Oak Common
08:08:26 [Train  1] enters block 3 for London Old Oak Common
08:08:46 [Train 17] enters block 9 for London Old Oak Common
08:08:47 [Train 15] enters block 12 for London Old Oak Common
08:08:51 [Train 19] enters block 6 for Lond

...

20:52:00 [Train 17] 4 passengers boarded
20:52:06 [Train 10] enters block 11 for London Old Oak Common
20:52:12 [Train 17] dep London Euston
20:52:12 [Train 17] enters block 1 for London Old Oak Common
20:52:13 [Train  8] enters block 14 for London Old Oak Common
20:52:24 [Train 15] enters block 4 for London Old Oak Common
20:52:34 [Train 13] enters block 7 for London Old Oak Common
20:52:53 [Train 11] enters block 10 for London Old Oak Common
20:53:06 [Train  9] enters block 13 for London Old Oak Common
20:53:23 [Train 16] enters block 3 for London Old Oak Common
20:53:30 [Train 14] enters block 6 for London Old Oak Common
20:53:49 [Train 12] enters block 9 for London Old Oak Common
20:54:00 [Train 18] drives to starting location
20:54:11 [Train 10] enters block 12 for London Old Oak Common
20:54:16 [Train 17] enters block 2 for London Old Oak Common
20:54:17 [Train  8] arr London Old Oak Common
20:54:17 [Train  8] 0 passengers boarded
20:54:17 [Train  8] arr final station
20:54:29 [T

21:16:00 [Train  5] 1 passengers boarded
21:16:03 [Train  5] dep London Euston
21:16:03 [Train  5] enters block 1 for London Old Oak Common
21:16:03 [Train 18] enters block 11 for London Old Oak Common
21:16:10 [Train 16] enters block 14 for London Old Oak Common
21:16:39 [Train  3] enters block 4 for London Old Oak Common
21:16:43 [Train  1] enters block 7 for London Old Oak Common
21:17:02 [Train 19] enters block 10 for London Old Oak Common
21:17:03 [Train 17] enters block 13 for London Old Oak Common
21:17:26 [Train  4] enters block 3 for London Old Oak Common
21:17:42 [Train  2] enters block 6 for London Old Oak Common
21:17:43 [Train  0] enters block 9 for London Old Oak Common
21:18:00 [Train  6] drives to starting location
21:18:07 [Train  5] enters block 2 for London Old Oak Common
21:18:08 [Train 18] enters block 12 for London Old Oak Common
21:18:14 [Train 16] arr London Old Oak Common
21:18:14 [Train 16] 0 passengers boarded
21:18:14 [Train 16] arr final station
21:18:44 [T

21:40:00 [Train 13] 6 passengers boarded
21:40:03 [Train  6] enters block 11 for London Old Oak Common
21:40:13 [Train  4] enters block 14 for London Old Oak Common
21:40:18 [Train 13] dep London Euston
21:40:18 [Train 13] enters block 1 for London Old Oak Common
21:40:21 [Train 11] enters block 4 for London Old Oak Common
21:40:34 [Train  9] enters block 7 for London Old Oak Common
21:40:54 [Train  5] enters block 13 for London Old Oak Common
21:40:59 [Train  7] enters block 10 for London Old Oak Common
21:41:26 [Train 12] enters block 3 for London Old Oak Common
21:41:39 [Train 10] enters block 6 for London Old Oak Common
21:41:52 [Train  8] enters block 9 for London Old Oak Common
21:42:00 [Train 14] drives to starting location
21:42:08 [Train  6] enters block 12 for London Old Oak Common
21:42:17 [Train  4] arr London Old Oak Common
21:42:17 [Train  4] 0 passengers boarded
21:42:17 [Train  4] arr final station
21:42:22 [Train 13] enters block 2 for London Old Oak Common
21:42:26 [T

22:24:11 [Train 18] 0 passengers boarded
22:24:11 [Train 18] arr final station
22:25:22 [Train 19] enters block 14 for London Old Oak Common
22:27:26 [Train 19] arr London Old Oak Common
22:27:26 [Train 19] 0 passengers boarded
22:27:26 [Train 19] arr final station


,departure,arrival,passengers,travel_time
0,21660.0,23400.0,0.0,1740.0
1,21840.0,23598.0,6.0,1758.0
2,22020.0,23778.0,6.0,1758.0
3,22200.0,23958.0,6.0,1758.0
4,22380.0,24138.0,6.0,1758.0
...,...,...,...,...
315,78360.0,80121.0,7.0,1761.0
316,78540.0,80310.0,10.0,1770.0
317,78720.0,80472.0,4.0,1752.0
318,78900.0,80652.0,4.0,1752.0


In [14]:
stats(results)

(1755.342857142866, 1600.0)

In [15]:
results.departure

0      21660.0
1      21840.0
2      22020.0
3      22200.0
4      22380.0
        ...   
315    78360.0
316    78540.0
317    78720.0
318    78900.0
319    79080.0
Name: departure, Length: 320, dtype: float64

In [16]:
results.to_csv (r'C:\Users\akile\Desktop\export_dataframe.csv', index = False, header=True)

In [17]:
#Monte Carlo Optimization method

def monte_carlo(n, xmin, xmax, ymin, ymax):
    x = [ random.randint(xmin, xmax) for i in range(n)]
    y = [ random.randint(ymin, ymax) for i in range(n)]
    xp = [ x[0] ]
    yp = [ y[0] ]
    
    #simulate
    res = runsimulation(x[0],y[0])
    avg, _ = stats(res)
    
    for i in range(1, len(x)):
        #simulate
        res = runsimulation(x[i],y[i])
        n_avg, _ = stats(res) 
        if n_avg < avg:
            xp += [x[i]]
            yp += [y[i]]
            avg = n_avg
    
    #simulate
    res = runsimulation(xp[-1],yp[-1])
    ag, _ = stats(res)
    
            
    return len(xp), xp[-1], yp[-1],ag

In [18]:
fer1=monte_carlo(10,1,20,1,15) #run simulation 100 times for min of 1 train and max of 20 trains, min of 1 signaling blocks and max of 15

06:00:00 [Train  0] drives to starting location
06:01:00 [Train  0] 2 passengers boarded
06:01:06 [Train  0] dep London Euston
06:01:06 [Train  0] enters block 1 for London Old Oak Common
06:03:00 [Train  1] drives to starting location
06:04:00 [Train  1] 3 passengers boarded
06:04:09 [Train  1] dep London Euston
06:06:00 [Train  2] drives to starting location
06:07:00 [Train  2] 4 passengers boarded
06:07:12 [Train  2] dep London Euston
06:09:00 [Train  3] drives to starting location
06:10:00 [Train  3] 11 passengers boarded
06:10:33 [Train  3] dep London Euston
06:12:00 [Train  4] drives to starting location
06:13:00 [Train  4] 3 passengers boarded
06:13:09 [Train  4] dep London Euston
06:15:00 [Train  5] drives to starting location
06:16:00 [Train  5] 3 passengers boarded
06:16:09 [Train  5] dep London Euston
06:18:00 [Train  6] drives to starting location
06:19:00 [Train  6] 8 passengers boarded
06:19:24 [Train  6] dep London Euston
06:21:00 [Train  7] drives to starting location
0

09:04:00 [Train  1] 8 passengers boarded
09:04:24 [Train  1] dep London Euston
09:06:00 [Train  2] drives to starting location
09:07:00 [Train  2] 3 passengers boarded
09:07:09 [Train  2] dep London Euston
09:09:00 [Train  3] drives to starting location
09:10:00 [Train  3] 4 passengers boarded
09:10:12 [Train  3] dep London Euston
09:12:00 [Train  4] drives to starting location
09:13:00 [Train  4] 8 passengers boarded
09:13:24 [Train  4] dep London Euston
09:15:00 [Train  5] drives to starting location
09:16:00 [Train  5] 3 passengers boarded
09:16:09 [Train  5] dep London Euston
09:18:00 [Train  6] drives to starting location
09:19:00 [Train  6] 4 passengers boarded
09:19:12 [Train  6] dep London Euston
09:21:00 [Train  7] drives to starting location
09:22:00 [Train  7] 4 passengers boarded
09:22:12 [Train  7] dep London Euston
09:24:00 [Train  8] drives to starting location
09:24:06 [Train  6] arr London Old Oak Common
09:24:06 [Train  6] 0 passengers boarded
09:24:06 [Train  6] arr 

11:58:00 [Train 19] 4 passengers boarded
11:58:12 [Train 19] dep London Euston
12:00:00 [Train  0] drives to starting location
12:01:00 [Train  0] 5 passengers boarded
12:01:15 [Train  0] dep London Euston
12:03:00 [Train  1] drives to starting location
12:04:00 [Train  1] 2 passengers boarded
12:04:06 [Train  1] dep London Euston
12:06:00 [Train  2] drives to starting location
12:07:00 [Train  2] 5 passengers boarded
12:07:15 [Train  2] dep London Euston
12:09:00 [Train  3] drives to starting location
12:10:00 [Train  3] 3 passengers boarded
12:10:09 [Train  3] dep London Euston
12:12:00 [Train  4] drives to starting location
12:13:00 [Train  4] 7 passengers boarded
12:13:21 [Train  4] dep London Euston
12:15:00 [Train  5] drives to starting location
12:16:00 [Train  5] 7 passengers boarded
12:16:21 [Train  5] dep London Euston
12:18:00 [Train  6] drives to starting location
12:18:06 [Train 12] arr London Old Oak Common
12:18:06 [Train 12] 0 passengers boarded
12:18:06 [Train 12] arr 

14:55:00 [Train 18] 4 passengers boarded
14:55:12 [Train 18] dep London Euston
14:57:00 [Train 19] drives to starting location
14:58:00 [Train 19] 4 passengers boarded
14:58:12 [Train 19] dep London Euston
15:00:00 [Train  0] drives to starting location
15:01:00 [Train  0] 6 passengers boarded
15:01:18 [Train  0] dep London Euston
15:03:00 [Train  1] drives to starting location
15:04:00 [Train  1] 7 passengers boarded
15:04:21 [Train  1] dep London Euston
15:06:00 [Train  2] drives to starting location
15:07:00 [Train  2] 5 passengers boarded
15:07:15 [Train  2] dep London Euston
15:09:00 [Train  3] drives to starting location
15:10:00 [Train  3] 4 passengers boarded
15:10:12 [Train  3] dep London Euston
15:12:00 [Train  4] drives to starting location
15:12:06 [Train 18] arr London Old Oak Common
15:12:06 [Train 18] 0 passengers boarded
15:12:06 [Train 18] arr final station
15:12:06 [Train 19] enters block 1 for London Old Oak Common
15:13:00 [Train  4] 5 passengers boarded
15:13:15 [T

17:52:00 [Train 17] 6 passengers boarded
17:52:18 [Train 17] dep London Euston
17:54:00 [Train 18] drives to starting location
17:55:00 [Train 18] 5 passengers boarded
17:55:15 [Train 18] dep London Euston
17:57:00 [Train 19] drives to starting location
17:58:00 [Train 19] 8 passengers boarded
17:58:24 [Train 19] dep London Euston
18:00:00 [Train  0] drives to starting location
18:01:00 [Train  0] 2 passengers boarded
18:01:06 [Train  0] dep London Euston
18:03:00 [Train  1] drives to starting location
18:04:00 [Train  1] 7 passengers boarded
18:04:21 [Train  1] dep London Euston
18:06:00 [Train  2] drives to starting location
18:06:06 [Train  4] arr London Old Oak Common
18:06:06 [Train  4] 0 passengers boarded
18:06:06 [Train  4] arr final station
18:06:06 [Train  5] enters block 1 for London Old Oak Common
18:07:00 [Train  2] 6 passengers boarded
18:07:18 [Train  2] dep London Euston
18:09:00 [Train  3] drives to starting location
18:10:00 [Train  3] 3 passengers boarded
18:10:09 [T

...

20:25:00 [Train  8] 5 passengers boarded
20:25:03 [Train  6] enters block 4 for London Old Oak Common
20:25:15 [Train  8] dep London Euston
20:25:15 [Train  8] enters block 1 for London Old Oak Common
20:25:31 [Train  1] enters block 12 for London Old Oak Common
20:25:34 [Train 19] enters block 15 for London Old Oak Common
20:25:46 [Train  3] enters block 9 for London Old Oak Common
20:25:58 [Train  7] enters block 3 for London Old Oak Common
20:26:01 [Train  5] enters block 6 for London Old Oak Common
20:26:29 [Train  0] enters block 14 for London Old Oak Common
20:26:35 [Train  2] enters block 11 for London Old Oak Common
20:26:53 [Train  4] enters block 8 for London Old Oak Common
20:26:59 [Train  6] enters block 5 for London Old Oak Common
20:27:00 [Train  9] drives to starting location
20:27:11 [Train  8] enters block 2 for London Old Oak Common
20:27:27 [Train  1] enters block 13 for London Old Oak Common
20:27:30 [Train 19] arr London Old Oak Common
20:27:30 [Train 19] 0 passeng

20:49:00 [Train 16] 4 passengers boarded
20:49:10 [Train  7] enters block 15 for London Old Oak Common
20:49:12 [Train 16] dep London Euston
20:49:12 [Train 16] enters block 1 for London Old Oak Common
20:49:25 [Train  9] enters block 12 for London Old Oak Common
20:49:52 [Train 11] enters block 9 for London Old Oak Common
20:49:58 [Train 13] enters block 6 for London Old Oak Common
20:50:10 [Train 15] enters block 3 for London Old Oak Common
20:50:23 [Train  8] enters block 14 for London Old Oak Common
20:50:32 [Train 10] enters block 11 for London Old Oak Common
20:50:50 [Train 12] enters block 8 for London Old Oak Common
20:50:50 [Train 14] enters block 5 for London Old Oak Common
20:51:00 [Train 17] drives to starting location
20:51:06 [Train  7] arr London Old Oak Common
20:51:06 [Train  7] 0 passengers boarded
20:51:06 [Train  7] arr final station
20:51:08 [Train 16] enters block 2 for London Old Oak Common
20:51:21 [Train  9] enters block 13 for London Old Oak Common
20:51:48 [T

21:13:00 [Train  4] 5 passengers boarded
21:13:03 [Train  2] enters block 4 for London Old Oak Common
21:13:15 [Train  4] dep London Euston
21:13:15 [Train  4] enters block 1 for London Old Oak Common
21:13:22 [Train 15] enters block 15 for London Old Oak Common
21:13:34 [Train 19] enters block 9 for London Old Oak Common
21:13:37 [Train 17] enters block 12 for London Old Oak Common
21:13:58 [Train  1] enters block 6 for London Old Oak Common
21:13:58 [Train  3] enters block 3 for London Old Oak Common
21:14:20 [Train 16] enters block 14 for London Old Oak Common
21:14:41 [Train  0] enters block 8 for London Old Oak Common
21:14:44 [Train 18] enters block 11 for London Old Oak Common
21:14:59 [Train  2] enters block 5 for London Old Oak Common
21:15:00 [Train  5] drives to starting location
21:15:11 [Train  4] enters block 2 for London Old Oak Common
21:15:18 [Train 15] arr London Old Oak Common
21:15:18 [Train 15] 0 passengers boarded
21:15:18 [Train 15] arr final station
21:15:30 [Tr

21:37:00 [Train 12] 4 passengers boarded
21:37:00 [Train 10] enters block 4 for London Old Oak Common
21:37:10 [Train  3] enters block 15 for London Old Oak Common
21:37:12 [Train 12] dep London Euston
21:37:12 [Train 12] enters block 1 for London Old Oak Common
21:37:34 [Train  5] enters block 12 for London Old Oak Common
21:37:34 [Train  7] enters block 9 for London Old Oak Common
21:37:58 [Train  9] enters block 6 for London Old Oak Common
21:38:16 [Train 11] enters block 3 for London Old Oak Common
21:38:23 [Train  4] enters block 14 for London Old Oak Common
21:38:32 [Train  8] enters block 8 for London Old Oak Common
21:38:35 [Train  6] enters block 11 for London Old Oak Common
21:38:56 [Train 10] enters block 5 for London Old Oak Common
21:39:00 [Train 13] drives to starting location
21:39:06 [Train  3] arr London Old Oak Common
21:39:06 [Train  3] 0 passengers boarded
21:39:06 [Train  3] arr final station
21:39:08 [Train 12] enters block 2 for London Old Oak Common
21:39:30 [Tr

22:06:12 [Train 12] 0 passengers boarded
22:06:12 [Train 12] arr final station
22:06:30 [Train 14] enters block 13 for London Old Oak Common
22:06:39 [Train 18] enters block 7 for London Old Oak Common
22:06:45 [Train 16] enters block 10 for London Old Oak Common
22:07:19 [Train 13] enters block 15 for London Old Oak Common
22:07:34 [Train 15] enters block 12 for London Old Oak Common
22:07:52 [Train 19] enters block 6 for London Old Oak Common
22:07:58 [Train 17] enters block 9 for London Old Oak Common
22:08:26 [Train 14] enters block 14 for London Old Oak Common
22:08:35 [Train 18] enters block 8 for London Old Oak Common
22:08:41 [Train 16] enters block 11 for London Old Oak Common
22:09:15 [Train 13] arr London Old Oak Common
22:09:15 [Train 13] 0 passengers boarded
22:09:15 [Train 13] arr final station
22:09:30 [Train 15] enters block 13 for London Old Oak Common
22:09:48 [Train 19] enters block 7 for London Old Oak Common
22:09:54 [Train 17] enters block 10 for London Old Oak Co

In [25]:
fer1

(2, 4, 15, 1755.834375)